<a href="https://www.kaggle.com/code/kruthikaushankar/malware-detector?scriptVersionId=245443144" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/network-malware-detection-connection-analysis/CTU-IoT-Malware-Capture-8-1conn.log.labeled.csv
/kaggle/input/network-malware-detection-connection-analysis/CTU-IoT-Malware-Capture-60-1conn.log.labeled.csv
/kaggle/input/network-malware-detection-connection-analysis/CTU-IoT-Malware-Capture-21-1conn.log.labeled.csv
/kaggle/input/network-malware-detection-connection-analysis/CTU-IoT-Malware-Capture-35-1conn.log.labeled.csv
/kaggle/input/network-malware-detection-connection-analysis/CTU-IoT-Malware-Capture-3-1conn.log.labeled.csv
/kaggle/input/network-malware-detection-connection-analysis/CTU-IoT-Malware-Capture-1-1conn.log.labeled.csv
/kaggle/input/network-malware-detection-connection-analysis/CTU-IoT-Malware-Capture-42-1conn.log.labeled.csv
/kaggle/input/network-malware-detection-connection-analysis/CTU-IoT-Malware-Capture-34-1conn.log.labeled.csv
/kaggle/input/network-malware-detection-connection-analysis/CTU-IoT-Malware-Capture-44-1conn.log.labeled.csv
/kaggle/input/network-

In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Step 1: Load and concatenate CSV files
folder_path = "/kaggle/input/network-malware-detection-connection-analysis"
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

df_list = []
for file in csv_files:
    df = pd.read_csv(os.path.join(folder_path, file), sep='|', low_memory=False)
    df_list.append(df)

data = pd.concat(df_list, ignore_index=True)

# Step 2: Drop unnecessary columns
drop_cols = ['uid', 'history', 'tunnel_parents', 'detailed-label', 'local_resp', 'local_orig', 'missed_bytes']
data.drop(columns=drop_cols, inplace=True, errors='ignore')

# Step 3: Drop rows with missing essential values
data.dropna(subset=['label', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes'], inplace=True)

# Step 4: Fix '-' string entries in numeric columns
for col in ['duration', 'orig_bytes', 'resp_bytes']:
    data[col] = data[col].replace('-', np.nan)  # Replace '-' with NaN
    data[col] = pd.to_numeric(data[col], errors='coerce')  # Convert to numeric
    data[col].fillna(0, inplace=True)  # Replace NaNs with 0

# Step 5: Encode categorical columns
data['proto'] = data['proto'].astype('category').cat.codes
data['service'] = data['service'].astype('category').cat.codes

# Step 6: Convert label to binary
data['label'] = data['label'].apply(lambda x: 0 if x == 'Benign' else 1)

# Step 7: Select features and labels
features = ['proto', 'service', 'duration', 'orig_bytes', 'resp_bytes']
X = data[features]
y = data['label']

# Step 8: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Step 9: Train Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)

# Step 10: Evaluate
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


/tmp/ipykernel_13/1187500509.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(0, inplace=True)  # Replace NaNs with 0


Accuracy: 0.7620521446459269
Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.52      0.60   1756032
           1       0.77      0.89      0.83   3246169

    accuracy                           0.76   5002201
   macro avg       0.75      0.71      0.72   5002201
weighted avg       0.76      0.76      0.75   5002201

